In [4]:
import keras
from keras import optimizers
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard
from tensorflow.keras.utils import to_categorical

batch_size    = 128
epochs        = 180
iterations    = 391
num_classes   = 10
log_filepath  = './lenet'

In [5]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('train:',len(x_train))
print('test :',len(x_test))
print('train_image :',x_train.shape)
print('train_label :',y_train.shape)
print('test_image :',x_test.shape)
print('test_label :',y_test.shape)


train: 50000
test : 10000
train_image : (50000, 32, 32, 3)
train_label : (50000, 1)
test_image : (10000, 32, 32, 3)
test_label : (10000, 1)


In [6]:
# one-hot encode
y_train = keras.utils.to_categorical(y_train, num_classes) # one-hot  (50000,10)
y_test = keras.utils.to_categorical(y_test, num_classes) # one-hot  (10000,10)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# data preprocessing
x_train /= 255
x_test /= 255


In [7]:
def build_model():
    model = Sequential()
    model.add(Conv2D(6, (5, 5), padding='valid', activation = 'relu', input_shape=(32,32,3)))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(16, (5, 5), padding='valid', activation = 'relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation = 'relu'))
    model.add(Dense(84, activation = 'relu'))
    model.add(Dense(10, activation = 'softmax'))
    sgd = optimizers.SGD(learning_rate=.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

# learning rate
def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 150:
        return 0.005
    return 0.001

In [8]:
# build network
model = build_model()
print(model.summary())

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# start traing
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,callbacks=cbks,
              validation_data=(x_test, y_test), shuffle=True)
# save model
model.save('lenet.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         456       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 6)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 120)               4

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
